# PointNet++ для семантической сегментации облака точек

Этот ноутбук настраивает окружение и запускает обучение модели PointNet++


## 1. Установка зависимостей


In [ ]:
# Установка всех необходимых библиотек
%pip install torch torchvision numpy scikit-learn tqdm matplotlib tensorboard -q


## 2. Клонирование репозитория или загрузка файлов


In [ ]:
# Вариант 1: Клонировать из GitHub (замените YOUR_USERNAME)
# !git clone https://github.com/YOUR_USERNAME/pointnet2-segmentation.git
# %cd pointnet2-segmentation

# Вариант 2: Загрузить файлы через вкладку Files в Colab
# Затем раскомментируйте:
# %cd /content/pointnet2-segmentation


## 3. Загрузка данных (опционально)


In [ ]:
from google.colab import files
import zipfile
import os

# Загрузите архив с данными (раскомментируйте если нужно)
# uploaded = files.upload()
# 
# Распакуйте архив
# with zipfile.ZipFile('data.zip', 'r') as zip_ref:
#     zip_ref.extractall('.')

# Или используйте синтетические данные (код автоматически их создаст)


## 4. Проверка данных


In [ ]:
from dataset import S3DISDataset
import torch

# Если у вас есть данные, укажите путь
# data_dir = '3011-20251217T195928Z-1-001'
# area = '3011'

# Или используйте синтетические данные
data_dir = '.'
area = 'synthetic'

dataset = S3DISDataset(data_dir, area=area, split='train', num_points=2048)
print(f'Dataset size: {len(dataset)}')
points, labels = dataset[0]
print(f'Points shape: {points.shape}, Labels shape: {labels.shape}')
print(f'Unique classes: {sorted(torch.unique(labels).tolist())}')


## 5. Обучение модели


In [ ]:
# Запуск обучения
!python train.py \
    --data_dir . \
    --area synthetic \
    --num_points 2048 \
    --batch_size 8 \
    --epochs 50 \
    --lr 0.001 \
    --num_classes 13 \
    --device cuda \
    --save_dir ./checkpoints \
    --log_dir ./logs


## 6. Мониторинг обучения (TensorBoard)


In [ ]:
# Запуск TensorBoard
%load_ext tensorboard
%tensorboard --logdir ./logs


## 7. Тестирование модели


In [ ]:
!python test.py \
    --data_dir . \
    --area synthetic \
    --checkpoint ./checkpoints/best_model.pth \
    --num_classes 13


## 8. Визуализация результатов


In [ ]:
!python visualize.py \
    --data_dir . \
    --area synthetic \
    --checkpoint ./checkpoints/best_model.pth \
    --num_samples 5 \
    --output_dir ./visualizations


## 9. Скачивание результатов


In [ ]:
from google.colab import files
import zipfile
import os

# Создаем архив с результатами
with zipfile.ZipFile('results.zip', 'w') as zipf:
    if os.path.exists('checkpoints'):
        for root, dirs, files_list in os.walk('checkpoints'):
            for file in files_list:
                zipf.write(os.path.join(root, file))
    if os.path.exists('visualizations'):
        for root, dirs, files_list in os.walk('visualizations'):
            for file in files_list:
                zipf.write(os.path.join(root, file))

# Скачиваем архив
files.download('results.zip')
